In [ ]:
import pandas as pd

file_path = '/Users/hitakiran/Desktop/FA24-Group6/Research/13k-recipes.csv'  
data = pd.read_csv(file_path)

#First 100 rows
data = data.head(100)

#Take out unwanted columns
data = data.drop(columns=['Unnamed: 0', 'Ingredients', 'Image_Name'])

#Clean the "Cleaned_Ingredients" column
cleaned_ingredients = []
for row in data['Cleaned_Ingredients']:
    #Convert the string representation of a list into an actual list
    ingredients_list = eval(row) 
    #Remove single quotes & whitespace
    cleaned_list = [ingredient.replace("'", "").strip() for ingredient in ingredients_list]
    #Put cleaned ingredients back into a single string
    cleaned_ingredients.append(", ".join(cleaned_list))

data['Cleaned_Ingredients'] = cleaned_ingredients
print(data)


                                                Title  \
0   Miso-Butter Roast Chicken With Acorn Squash Pa...   
1                     Crispy Salt and Pepper Potatoes   
2                         Thanksgiving Mac and Cheese   
3                  Italian Sausage and Bread Stuffing   
4                                        Newton's Law   
..                                                ...   
95           Corn and Chickpea Bowls With Miso Tahini   
96                           Stone Fruit Custard Tart   
97  Plátanos Horneados con Crema y Queso (Baked B...   
98              Tamale Pie with Fresh Tomato and Corn   
99  Charred-Peach Panzanella With Pickled Pepper V...   

                                         Instructions  \
0   Pat chicken dry with paper towels, season all ...   
1   Preheat oven to 400°F and line a rimmed baking...   
2   Place a rack in middle of oven; preheat to 400...   
3   Preheat oven to 350°F with rack in middle. Gen...   
4   Stir together brown sugar 

In [11]:
measurements = ["g", "kg", "lb", "oz", "cup", "cups", "tsp", "tbsp", "tablespoon", "teaspoon", "ml", "l", "liter", "liters", "dash", "pinch", "pound", "ounce"]
fractions = ["½", "¼", "¾", "⅓", "⅔"] 

cleaned_ingredients_final = []
for row in data['Cleaned_Ingredients']:
    ingredients_list = row.split(", ") #Split ingredients
    cleaned_list = []
    
    for ingredient in ingredients_list: #Remove numbers and fractions
        words = ingredient.split()  #Split into individual words
        filtered_words = [
            word for word in words 
            if not (word.isdigit() or word in fractions or word.lower() in measurements)
        ]
        #Putting filtered words back into a cleaned ingredient string
        cleaned_list.append(" ".join(filtered_words).strip())
    
    #Combine cleaned ingredients to string
    cleaned_ingredients_final.append(", ".join(cleaned_list))

# Update the column
data['Cleaned_Ingredients'] = cleaned_ingredients_final
data

,Title,Instructions,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...","whole chicken, kosher salt, divided, plus more..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,"large egg whites, new potatoes about inch in d..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,"evaporated milk, whole milk, garlic powder, on..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,"to round Italian loaf, cut into inch cubes, ta..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,"dark brown sugar, hot water, bourbon, fresh le..."
...,...,...,...
95,Corn and Chickpea Bowls With Miso Tahini,"Purée jalapeños, ginger, garlic, cilantro, lim...","small jalapeos, seeded, chopped, inch piece gi..."
96,Stone Fruit Custard Tart,Preheat an oven to 375°F. Cook butter in a sma...,"stick unsalted butter, raw pistachios, divided..."
97,Plátanos Horneados con Crema y Queso (Baked B...,Preheat the oven to 400°F.\nPlace the whole ba...,"heirloom bananas or small plantains, ripe but ..."
98,Tamale Pie with Fresh Tomato and Corn,"Preheat oven to 450°F. Heat an 8"" cast-iron sk...","fresh chorizo, casings removed about links, or..."
